# IMPORT

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import tensorflow as tf
import numpy as np
import pickle

%matplotlib inline
from matplotlib import pyplot as plt

print(tf.__version__)

1.0.1


# PARAMETERS

In [3]:
#dir
DATA_PATH = './cifar10/cifar-10-batches-py/' #pickle

print(DATA_PATH)

#param 
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
IMAGE_SIZE = IMAGE_HEIGHT * IMAGE_WIDTH
N_CHANNEL = 3
LABEL_BYTE = 1

BATCH_SIZE = 128

EPOCHS = 1000

LEARNING_RATE = 0.1
LR_DECAY_RATE = 0.5
WEIGHT_DECAY = 0.0005
MOVING_AVERAGE_DECAY = 0.9999

NUM_EXAMPLES_PER_EPOCH = 50000 #NUMBER OF TRAIN DATA SET
NUM_BATCHES_PER_EPOCH = NUM_EXAMPLES_PER_EPOCH / BATCH_SIZE
NUM_EPOCHS_PER_DECAY = 350
NUM_TESTDATA_PER_EPOCH = 10000
NUM_TEST_BATCHES_PER_EPOCH = NUM_TESTDATA_PER_EPOCH / BATCH_SIZE

LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(LIST_CLASS)

print("input image size : {}".format(IMAGE_SIZE))
print("image channel : {}".format(N_CHANNEL))
print("batch size : {}".format(BATCH_SIZE))
print("num of class : {}".format(N_CLASSES))
print("training epochs : {}".format(EPOCHS))
print("learning rate : {}".format(LEARNING_RATE))
print("learning decay rate : {}".format(LR_DECAY_RATE))

./cifar10/cifar-10-batches-py/
input image size : 1024
image channel : 3
batch size : 128
num of class : 10
training epochs : 1000
learning rate : 0.1
learning decay rate : 0.5


# DATA READ (CIFAR-10)

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
def bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def int64_feature(value):
    """Wrapper for inserting int64 features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [ ]:
filenames = [os.path.join(DATA_PATH, 'data_batch_%d' % i) for i in range(1, 6)]
filenames.append(os.path.join(DATA_PATH, 'test_batch'))
print(filenames)

tf_filenames_train = [os.path.join(DATA_PATH, 'data_batch_%d.tfrecords' % i) for i in range(1, 6)]
tf_filenames_test = [(os.path.join(DATA_PATH, 'test_batch.tfrecords'))]
print(tf_filenames)

In [ ]:
for path in filenames:
    #tfrecord가 만들어진 경우 넘어감
    tf_path = path + '.tfrecords'
    if os.path.exists(tf_path):
        continue
    
    #파일이 없는 경우 에러
    if not os.path.exists(path):
        raise ValueError('Failed to find file: ' + path)
    
    dict = unpickle(path)
    writer = tf.python_io.TFRecordWriter(tf_path)
    images = dict[b'data']
    labels = dict[b'labels']
    
    for i in range(images.shape[0]):
        image = images[i,:]
        label = labels[i]
        feature = {'cifar/image' : bytes_feature(value=[image.tostring()]), 
                   'cifar/label' : int64_feature(value=[label])}
        example = tf.train.Example(features=tf.train.Features(features=feature))
        serialized = example.SerializeToString()
        writer.write(serialized)

# INPUT PIPELINE FROM TFRECORDS

In [ ]:
def input_batches(num_threads=8, test=False, distorted=False):
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH * min_fraction_of_examples_in_queue)
    capacity = batch_size * 3 + min_queue_examples #queue capacity 
    
    #test or train
    tf_files = None
    if test:
        tf_files = tf_filenames_test
    else:
        tf_files = tf_filenames_train
        
    filename_queue = tf.train.string_input_producer(tf_files, shuffle=True)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    feature = {'cifar/image' : tf.FixedLenFeature(shape=[], dtype=tf.string),
              'cifar/label' : tf.FixedLenFeature(shape=[], dtype=tf.int64)}
    
    features = tf.parse_single_example(serialized_example, features=feature)
    
    image = tf.reshape(tf.decode_raw(features['cifar/image'], tf.uint8), [IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNEL])
    
    image = tf.cast(image, tf.float32)
    label = features['cifar/label']
    
    if distored:
        image = tf.random_crop(image, [IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNEL]) #사실상 필요 없음
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
        #몇가지 더 추가해 볼 것.
        
        
    #standardization 
    image = tf.image.per_image_standardization(image)
    
    image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size=BATCH_SIZE)
    
    return image_batch, label_batch

# MODEL ( RES NET )

#### Residual 32 - 7x7 64 /2 > pool /2 > 3x3 64 > 3x3 64 > concat > 3x3 64 > 3x3 64 > concat > 3x3 64 > 3x3 64 > concat > 3x3 128 /2 > 3x3 128 > 3x3 128 > 3x3 128 > concat > 3x3 128 > 3x3 128 > concat > 3x3 128 > 3x3 128 > concat > 3x3 256 /2 > 3x3 256 > 3x3 256 >  3x3 256 > concat > 3x3 256 >  3x3 256 > concat > 3x3 256 >  3x3 256 > concat > 3x3 256 >  3x3 256 > concat > 3x3 256 >  3x3 256 > concat > 3x3 512 /2 > 3x3 512 > 3x3 512 > 3x3 512 > concat > 3x3 512 > 3x3 512 > concat > avg pool > fc 1000
http://felixlaumon.github.io/2015/01/08/kaggle-right-whale.html


#### cifar10-residual - 3x3 16 > 3x3 16 > 3x3 16 > concat > 3x316 > 3x3 16 > concat > max pool? > 3x3 32 > 3x3 32 > concat > 3x3 32 > 3x3 32 > concat > max pool? > 3 x 3 64 > 3 x 3 64 > concat  > 3 x 3 64 > 3 x 3 64 > concat >  avg pool > softmax
http://laonple.blog.me/220770760226

#### bottle neck > 1x1 out/4 > 3x3 out/4 > 1x1 > out

## 참고해볼 것 http://florianmuellerklein.github.io/wRN_vs_pRN/
## https://github.com/tensorflow/models/blob/master/resnet/resnet_model.py


In [ ]:
def bottleneck(inputs, output_depth, is_training=False, reuse=False, scope=None):
    with tf.variable_scope(scope, 'bottle_neck', reuse=reuse):
        _output = tf.layers.conv2d(inputs, int(output_depth/4), (1,1), padding='SAME', 
                                   activation=tf.nn.relu, use_bias=False,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(WEIGHT_DECAY), 
                                   name='squeezed_conv1x1')
        _output = tf.layers.conv2d(_output, int(output_depty/4), (3,3), padding='SAME', 
                                   activation=tf.nn.relu, use_bias=False,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(WEIGHT_DECAY), 
                                   name='squeezed_conv3x3')
        _output = tf.layers.conv2d(_output, output_depty, (1,1), padding='SAME', 
                                   activation=tf.nn.relu, use_bias=False,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(WEIGHT_DECAY), 
                                   name='expanded_conv1x1')
        _output = tf.layers.batch_normalization(_output, training=is_training, name='batch_norm_bottleneck')
        return _output

def resmodule(inputs, output_depth, is_training=False, reuse=False, scope=None):
    with tf.variable_scope(scope, 'resnet_module', reuse=reuse):
        _net = bottleneck(inputs, int(output_depth/2), is_training=is_training, reuse=reuse, name='bottleneck1')
        print(net.name, net.get_shape())
        _net = bottleneck(_net, int(output_depth/2), is_training=is_training, reuse=reuse, name='bottleneck2')
        print(net.name, net.get_shape())
        _output = tf.concat([net, inputs], axis=3)
        return _output

In [ ]:
def RES(images, num_classes, is_training=False, reuse=False, scope='ResNet'):
    with tf.variable_scope(scope, 'resnet', reuse=reuse):
        net = tf.layers.conv2d(images, 16, (3,3), padding='SAME',
                               activation=tf.nn.relu, use_bias=False,
                               kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(WEIGHT_DECAY), 
                               name='1st_conv')
        print(net.name, net.get_shape())
        net = resmodule(net, 32, is_training=is_training, reuse=reuse)
        print(net.name, net.get_shape())
        net = tf.layers.max_pooling2d(net, (2,2), strides=(2,2), padding='SAME', name='maxpool1')
        print(net.name, net.get_shape())
        net = resmodule(net, 64, is_training=is_training, reuse=reuse)
        print(net.name, net.get_shape())
        net = tf.layers.max_pooling2d(net, (2,2), strides=(2,2), padding='SAME', name='maxpool2')
        print(net.name, net.get_shape())
        net = resmodule(net, 128, is_training=is_training, reuse=reuse)
        print(net.name, net.get_shape())
        net = tf.layers.average_pooling2d(net, (2,2), strides=(2,2), padding='SAME', name='avgpool')
        print(net.name, net.get_shape())
        net = tf.layers.conv2d(net, num_classes, (1,1), name='flatten')
        print(net.name, net.get_shape())
        logits = tf.squeeze(net, (1,2), name='logits')
        print(logits.name, logits.get_shape())
        return logits

# MODEL ( VGG NET )

#### VGG 19 - 3x3 64 > 3x3 64 > pool /2 > 3x3 128 >  3x3 128 > pool /2 >  3x3 256 > 3x3 256 > 3x3 256 > 3x3 256 > pool /2 > 3x3 512 > 3x3 512 > 3x3 512 > 3x3 512 > pool /2 > 3x3 512 > 3x3 512 > 3x3 512 > 3x3 512 > pool /2 > fc 4096 > fc 4096 > fc 1000 > classes
#### relu >>>>> softmax

#### VGG 16 3x3 64 > 3x3 64 > pool /2 > 3x3 128 >  3x3 128 > pool /2 > 3x3 256 > 3x3 256 > 3x3 256 > pool /2 > 3x3 512 > 3x3 512 > 3x3 512 > pool /2 > 3x3 512 > 3x3 512 > 3x3 512 > pool /2 > fc 4096 > fc 4096 > fc 1000 > classes
#### relu >>>>> softmax

#### VGG for CIFAR-10 > 3x3 16 > 3x3 16 > pool /2 > 3x3 32 >  3x3 32 > 3x3 32 > pool /2 > 3x3 64 > 3x3 64 > 3x3 64 > 3x3 64 > pool /2 > fc 2048 > fc 2048 >  fc 500 > softmax
#### relu >>>>> softmax

In [ ]:
def VGG(images, reuse=False):
   

# LOSS, TRAIN(EVAL) OP

In [ ]:
def loss(logits, labels, total=True):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits, name='cross_entropy')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='mean_cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean) #loss +++ 
    _loss = cross_entropy_mean
    
    if total:
        print("adding total loss", len(tf.get_collection('losses')))
        _loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
    return _loss


In [ ]:
def _add_total_loss_summary(total_loss, test=False):
    emaloss = tf.train.ExponentialMovingAverage(decay=0.9, name='avg')
    losses = []
    if not test:
        losses = tf.get_collection('losses')

    loss_average_op = emaloss.apply(losses + [total_loss])
    for ls in losses + [total_loss]:
        tf.summary.scalar(ls.op.name + '_raw_', ls)
        tf.summary.scalar(ls.op.name + 'ema', emaloss.average(ls))
    return loss_average_op

#http://cafe.naver.com/soynature/1942
#ema / bn 

In [ ]:
def trainop(global_step, topk=1):
    decay_steps = int(NUM_BATCHES_PER_EPOCH * NUM_EPOCHS_PER_DECAY)
    lr = tf.train.exponential_decay(LEARNING_RATE, global_step, decay_steps, LR_DECAY_RATE, staircase=True)
    tf.summary.scalar('learning_rate', lr)
    
    images, labels = input_batches(num_threads=8)
    logits = RES(images=images, is_training=True, num_classes=N_CLASSES)
    totalloss = loss(logits, labels)
    predictions = tf.nn.in_top_k(logits, labels, topk)
    truecount = tf.cast(predictions, tf.float32)
    accuracy = tf.reduce_mean(truecount)
   
    loss_average_op = _add_total_loss_summary(totalloss)
    
    optimizer = tf.train.AdamOptimizer(lr)

    with tf.control_dependencies([loss_average_op]): #gradient clip ... 등등을 위해서 보통 나눔
        gradsvars = optimizer.compute_gradients(totalloss)
        apply_grads_op = optimizer.apply_gradients(gradsvars, global_step=global_step)
    for grad, var in gradsvars:
        tf.summary.histogram(var.op.name, var)
        if grad is not None:
            tf.summary.histogram(var.op.name + '/grads', grad)
    
    ema_variables = tf.train.ExponentialMovingAverage(decay=MOVING_AVERAGE_DECAY, global_step)
    vars_averages_op = ema_variables.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_grads_op, vars_averages_op]):
        train_op = tf.no_op(name='train')
    
    return train_op, totalloss, accuracy

In [ ]:
def evalop(topk=1):
    images, labels = input_batches(2, test=True)
    logits = RES(images=images, is_training=False, num_classes=N_CLASSES, reuse=True)
    evalloss = loss(logits, labels, total=False)
    predictions = tf.nn.in_top_k(logits, labels, topk)
    truecount = tf.cast(predictions, tf.float32)
    accuracy = tf.reduce_mean(truecount)
    return evalloss, accuracy

# TRAIN

In [ ]:
def train():
    with tf.Graph().as_default():
        global_step = tf.contrib.framework.get_or_create_global_step() #global step
        train_op, totalloss, trainaccuracy = trainop(global_step) #train op
        evalloss, evalaccuracy = evalop() #eval op
        class _LoggerHook(tf.train.SessionRunHook): #for문이랑 비슷한 역할임. > 세션훅 상속
            def begin(self): #가장 처음 한번만 실행 
                self._step = -1
            def before_run(self, run_context): #포문 
                self._step += 1
                self._start_time = time.time()
                if self._step % 100 == 0:
                    return tf.train.SessionRunArgs([totalloss, trainaccuracy, evalloss, evalaccuracy])
                if self._step % 10 == 0:
                    return tf.train.SessionRunArgs([totalloss, trainaccuracy])
                else:
                    return tf.train.SessionRunArgs([totalloss])
            def after_run(self, run_context, run_values): #끝? 리턴 후에만 실행??? 
                duration = time.time() - self._start_time
                loss_value = run_values.results[0]
                if self._step % 10 == 0:
                    trainaccuracy = run_values.results[1]
                    num_examples_per_step = Flags.batch_size
                    examples_per_sec = num_examples_per_step / duration
                    sec_per_batch = float(duration)

                    format_str = ('%s: step %d, loss = %.2f  accuracy = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print (format_str % (time.time(), self._step, loss_value, trainaccuracy, 
                               examples_per_sec, sec_per_batch))
                if self._step % 100 == 0:
                    evalloss = run_values.results[2]
                    accuracy = run_values.results[3]
                    print('eval result: eval_loss: %.2f  accuracy: %.2f'  % (evalloss, accuracy))
                    print('')
            
        with tf.train.MonitoredTrainingSession( #모니터링세션
            checkpoint_dir=Flags.train_dir, 
            hooks=[tf.train.StopAtStepHook(last_step=Flags.max_steps), tf.train.NanTensorHook(totalloss), _LoggerHook()], #훅들
            config=tf.ConfigProto(log_device_placement=Flags.log_device_placement)) as mon_sess:
            while not mon_sess.should_stop():
                mon_sess.run(train_op)